In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_parquet("data/fhv_tripdata_2021-01.parquet")

In [5]:
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037
...,...,...,...,...,...,...,...
1154107,B03266,2021-01-31 23:43:03,2021-01-31 23:51:48,7.0,7.0,None,B03266
1154108,B03284,2021-01-31 23:50:27,2021-02-01 00:48:03,44.0,91.0,None,
1154109,B03285,2021-01-31 23:13:46,2021-01-31 23:29:58,171.0,171.0,None,B03285
1154110,B03285,2021-01-31 23:58:03,2021-02-01 00:17:29,15.0,15.0,None,B03285


In [7]:
df['duration'] = df.dropOff_datetime - df.pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [9]:
df.duration.mean()

19.167224093791006

In [10]:
df_drop = df[(df.duration >= 1) & (df.duration <= 60)]

In [14]:
print(df_drop.shape, df.shape)
print("Dropped: ", df.shape[0] - df_drop.shape[0])

(1109826, 8) (1154112, 8)
Dropped:  44286


In [19]:
categorical = ['PUlocationID', 'DOlocationID']
numerical = ['trip_distance']
# df[categorical] = df[categorical].astype(str)
df_drop = df_drop[categorical].fillna(-1)

In [22]:
(df_drop['PUlocationID'] == -1).sum() / df_drop['PUlocationID'].shape[0]

0.8352732770722617

In [55]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

sns.distplot(y_pred, label="pred", bins=50)
sns.distplot(y_train, label="actual", bins=50)

mean_squared_error(y_pred, y_train, squared=False)

In [60]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [61]:
df_train = read_dataframe("data/fhv_tripdata_2021-01.parquet")
df_val = read_dataframe("data/fhv_tripdata_2021-01.parquet")

In [62]:
len(df_train), len(df_val)

(73908, 61921)

In [63]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient='records')

X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [64]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [66]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_val = lr.predict(X_val)


display(mean_squared_error(y_pred_train, y_train, squared=False))
display(mean_squared_error(y_pred_val, y_val, squared=False))

9.775464298045627

10.473871278500406

In [77]:
lr = Ridge()
lr.fit(X_train, y_train)

y_pred_val = lr.predict(X_val)

display(mean_squared_error(y_pred_val, y_val, squared=False))

10.860753390350608

In [80]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']